In [238]:
from PIL import Image, ImageDraw
import numpy as np
from enum import IntEnum
import world_object
from abc import ABC, abstractmethod
import random
import math

In [241]:
COLORS = {
    "red": '#FF000000',
    "green": '#00FF00',
    "blue": '#0000FF',
    "purple": '#A020F0',
    "yellow": '#FFFF00',
    "grey": '#808080',
}

SHAPES = {
    "TRIANGLE":"triangle",
    "SQUARE":"square",
    "CIRCLE":"circle",
    "RECTANGLE":"rectangle"
}

# Map of object type to integers
OBJECT_TO_IDX = {
    "empty": 0,
    "agent": 1,
    "goal": 8,
    "object":10,
}

IDX_TO_OBJECT = {v: k for k, v in OBJECT_TO_IDX.items()}

# Map of agent direction indices to vectors
DIR_TO_VEC = [
    # Pointing right (positive X)
    np.array((1, 0)),
    # Down (positive Y)
    np.array((0, 1)),
    # Pointing left (negative X)
    np.array((-1, 0)),
    # Up (negative Y)
    np.array((0, -1)),
]

WIDTH = 200
HEIGHT = 200
GRID_SIZE = 20

'empty'

In [3]:
X = [[1,2,3],
     [1,2,3],
     [1,2,3]
    ]

In [4]:
class Actions(IntEnum):
    # Turn left, turn right, move forward
    left = 0
    right = 1
    forward = 2

In [288]:
class Shape:
    '''
        Attributes
        ----------
        distance_from_cooridinates : int
            How far the center should be from the points
        color : str
            A hex representation of the color that triangle should be
        fill : str
            Whether the object should be filled or not
            
        Methods
        -------
        init(self, size=int, color=str, fill=Bool, sound=None)
            Initializes the class
        draw(self, world=ImageDraw, x=int, y=int)
            Draws the triangle to the given Image at the x,y coordinates
        get_coordinates(self, x=int, y=int)
            Given a centerpoint, calculates the points of the triangle
    '''
    def __init__(self, size, color, fill = False):
        self.distance_from_coordinate = (size/2)
        self.color = color
        self.fill = fill
        
    @abstractmethod
    def draw(self):
        pass
    
    @abstractmethod
    def get_coordinates():
        pass
        
class Triangle(Shape):
    def __init__(self, size, color, fill = False):
        Shape.__init__(size, color, fill)
        self.shape = "TRIANGLE"
    
    #Draw a triangle around this point
    def draw(self, world, x, y):
        ImageDraw.ink = self.color
        bottom_left,bottom_right,top = self.get_coordinates(x,y)
        world.polygon([bottom_left, bottom_right, top], self.color, outline="black")
        
    def get_coordinates(self, x, y):
        x1 = x - self.distance_from_coordinate
        x2 = x + self.distance_from_coordinate
        x3 = x
        y1 = y + self.distance_from_coordinate
        y2 = y + self.distance_from_coordinate
        y3 = y - self.distance_from_coordinate
        
        return (x1, y1), (x2, y2), (x3, y3)

class Rectangle(Shape):
    def __init__():
        super().__init__()
        self.shape = "RECTANGLE"
        
    #Draw a triangle around this point
    def draw(self, world, x, y):
        ImageDraw.ink = self.color
        top_left, bottom_right  = self.get_coordinates(x,y)
        print(top_left, bottom_right)
        world.rectangle([top_left, bottom_right], self.color, outline="black")
        
    def get_coordinates(self, x, y):
        height = self.distance_from_coordinate/2
        x1, y1 = x - self.distance_from_coordinate, y - height
        x2, y2 = x + self.distance_from_coordinate, y + height
        return (x1, y1), (x2, y2)

class Square(Shape):
    def __init__():
        super().__init__()
        self.shape = "SQUARE"
    
    def draw(self, world, x, y):
        ImageDraw.ink = self.color
        top_left, bottom_right = self.get_coordinates(x,y)
        world.rectangle([top_left, bottom_right], self.color, outline="black")
        
    def get_coordinates(self, x, y):
        x1, y1 = x - self.distance_from_coordinate, y - self.distance_from_coordinate
        x2, y2 = x + self.distance_from_coordinate, y + self.distance_from_coordinate
        return (x1, y1), (x2, y2)

class Circle(Shape):
    def __init__():
        super().__init__()
        self.shape = "CIRCLE"

    def draw(self, world, x, y):
        ImageDraw.ink = self.color
        bottom_left, top_right = self.get_coordinates(x,y)
        world.ellipse([bottom_left, top_right], fill = self.color, outline="black")
        
    def get_coordinates(self, x, y):
        x1, y1 = x - self.distance_from_coordinate, y - self.distance_from_coordinate
        x2, y2 = x + self.distance_from_coordinate, y + self.distance_from_coordinate
        return (x1, y1), (x2, y2) 


    
class Agent:
    def __init__(self, world, starting_loc, shape = Triangle, color='#FFFF00'):
        self.row, self.col = starting_loc
        self.shape = shape(height, color)
            
    def update_loc(row, col):
        self.row = row
        self.col = col
            
    def draw(self, world, x, y):
        self.shape.draw(world, x, y)
    
    
class GridWorld:
    def __init__(self, size, background_color='#FFFFFF'):
        self.size = size
        self.x_offset = int(abs((WIDTH - size*GRID_SIZE)/2))
        self.y_offset = int(abs((HEIGHT - size*GRID_SIZE)/2))
        self.world = np.empty(shape = (size, size), dtype=object)
        #self.__setup_initial_locations()

        self.agent = Agent(self, (10, 10))
        
    def __setup_initial_locations(self):
        #Makes min objects 1, so goal can be placed
        total_objects = math.ceil((self.size**self.size)/3)
        agent = HelperMethods.find_free_space(self.world, 'agent')
        goal = HelperMethods.find_free_space(self.world, 'goal')
        while total_objects > 0:
            HelperMethods.find_free_space(self.world, 'object')
    
    def draw(self, draw):
        step_size = GRID_SIZE 

        y_start = self.y_offset
        y_end = self.y_offset + (step_size*self.size)
        x_start = self.x_offset
        x_end = self.x_offset + (step_size*self.size)
        
        for x in range(y_start, y_end + 1, step_size):
            line = ((x, y_start), (x, y_end))
            draw.line(line, fill=128)

        for y in range(x_start, x_end + 1, step_size):
            line = ((x_start, y), (x_end, y))
            draw.line(line, fill=128)
            
        return image, step_size

    def draw_objects(self):
        centroid = GRID_SIZE/2
        for row in range(0, self.size):
            for col in range(0, self.size):
                world_object = self.world[row, col]
                if world_object != 0:
                    
    
class HelperMethods:
    def find_free_space(array, object_to_place):
        if object_to_place == "object":
            pass
        count = 0
        row, col = None, None
        while row is None and col is None or array[row, col] != 0:
            if(count > array.size):
                return (-1, -1)
            row = random.randint(0, array.shape[0] - 1)
            col = random.randint(0, array.shape[1] - 1)
            count+=1
        array[row, col] = OBJECT_TO_IDX[object_to_place]
        return (row, col)
        
    def get_random_shape_and_color(world):
        agent_shape = world.agent.shape
        agent_color = world.agent.shape.color
        return agent_shape
        
        

IndentationError: expected an indented block after 'if' statement on line 159 (3387611250.py, line 162)

In [274]:
world2 = GridWorld(4)

x = HelperMethods.get_random_shape_and_color(world2)
print(x.shape)

AttributeError: 'Triangle' object has no attribute 'shape'

In [142]:
# Test objects are being drawn correctly
image = Image.new(mode='RGB', size=(WIDTH, HEIGHT), color=(255,255,255, 0))
draw = ImageDraw.Draw(image)

world = GridWorld(4)

image, step_size = world.draw(draw)
size = step_size/2
draw = ImageDraw.Draw(image)

tri = Triangle(size, COLORS['yellow'])
tri.draw(draw, 90, 110)

circ = Circle(size, COLORS['blue'])
circ.draw(draw, 10, 10)

rec = Rectangle(size, COLORS['green'])
rec.draw(draw, 30, 30)

square = Square(size, COLORS['grey'])
square.draw(draw, 70, 70)

del draw

image.show()

60
(25.0, 27.5) (35.0, 32.5)


In [293]:
image = Image.new(mode='RGB', size=(WIDTH, HEIGHT), color=(255,255,255, 0))
draw = ImageDraw.Draw(image)
world = GridWorld(6)
world.draw(draw)
world.draw_objects()
image.show()



TypeError: Triangle.__init__() takes 0 positional arguments but 3 were given